In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

np.random.seed(59)
sns.set_theme(style="ticks", context="poster", font_scale=1.4)


from conformal_tree import ConformalTreeClassification
from demo_util import gen_sin_data, split_conformal

# Example data

We demonstrate conformal tree for classification on the dermatology data example

The data is taken from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/33/dermatology)

In [2]:
data_path = './data/dermatology.data'
df = pd.read_csv(data_path, header=None)

columns = [
    'erythema', 'scaling', 'definite_borders', 'itching', 'koebner_phenomenon', 'polygonal_papules',
    'follicular_papules', 'oral_mucosal_involvement', 'knee_and_elbow_involvement', 'scalp_involvement',
    'family_history', 'melanin_incontinence', 'eosinophils_in_the_infiltrate', 'PNL_infiltrate',
    'fibrosis_of_the_papillary_dermis', 'exocytosis', 'acanthosis', 'hyperkeratosis', 'parakeratosis',
    'clubbing_of_the_rete_ridges', 'elongation_of_the_rete_ridges', 'thinning_of_the_suprapapillary_epidermis',
    'spongiform_pustule', 'munro_microabcess', 'focal_hypergranulosis', 'disappearance_of_the_granular_layer',
    'vacuolisation_and_damage_of_basal_layer', 'spongiosis', 'saw-tooth_appearance_of_retes',
    'follicular_horn_plug', 'perifollicular_parakeratosis', 'inflammatory_monoluclear_inflitrate',
    'band-like_infiltrate', 'age', 'class'
]

df.columns = columns

df.replace('?', pd.NA, inplace=True)
df['age'] = pd.to_numeric(df['age'], errors='coerce')

mean_age = df['age'].mean()
df['age'] = df['age'].fillna(mean_age)

df

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55.0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8.0,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26.0,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40.0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25.0,4
362,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36.0,4
363,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28.0,3
364,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50.0,3


The data contains clinical features, which are observed by a practitioner in a clinical setting, as well as histopathological features, which are studied through blood samples. In this case, we restrict to only clinical features, and try to predict the class of disease from 6 possibilities

In [3]:
feature_columns = [
    'erythema', 'scaling', 'definite_borders', 'itching', 'koebner_phenomenon', 'polygonal_papules',
    'follicular_papules', 'oral_mucosal_involvement', 'knee_and_elbow_involvement', 'scalp_involvement',
    'family_history', 'age'
]
clinical_vals = df[feature_columns].values
clinical_labels = df["class"].values

In this case, the black-box model is using an LLM (GPT-4) to classify the disease based on clinical features. For this demonstration, we provide the response data for each observation in the dermatology dataset. For each sample, GPT assigned probabilities to the six classes. This data is stored as dictionaries in `data/dermdata.json`.

In [4]:
import json
with open("./data/dermdata.json", "r") as f:
    output_data = json.load(f)

Here is an example of what the predictions for the first sample look like

In [5]:
print(output_data[0])

{'psoriasis': 0.25, 'seboreic dermatitis': 0.3, 'lichen planus': 0.05, 'pityriasis rosea': 0.1, 'cronic dermatitis': 0.25, 'pityriasis rubra pilaris': 0.05}


For our experiment, we split the available data into calibration data and test data.

In [6]:
n = len(df)
indices = np.arange(0, n)
calib_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

y_calib_pred = np.array(output_data)[calib_idx]
y_test_pred = np.array(output_data)[test_idx]

X_calib = clinical_vals[calib_idx]
y_calib = clinical_labels[calib_idx]
X_test = clinical_vals[test_idx]

For our classification example, we use the function $S(x,y)=1-f(x)_y$ for conformity scores. In this case, $f:\mathcal{X}\times \mathcal{Y}\to[0,1]$ maps a datapoint $x$ and a label $y$ to probability that the model assigns to every class except $y$ for sample $x$.

In [7]:
conf_scores = []
for i,sp in enumerate(y_calib_pred):
    true_idx = clinical_labels[calib_idx][i]-1
    conf = 1.0 - float(list(sp.values())[true_idx])
    conf_scores.append(conf)
conf_scores = np.array(conf_scores)

n = len(calib_idx)
alpha = 0.1
conf_quant = np.quantile(conf_scores, np.ceil((1 - alpha)*(n+1))/n)

We define the domain for the robust tree below, converting the ordinal range $(0,1,2,3)$ to the interval $[-1,4]$. One can verify that this is equivalent to splitting the ordinal range $\{0,1,2,3\}$ dyadically.

The `calibrate` method performs the exact same logic for a classification example as a regression example, as it acts on the conformity scores.

In [8]:
domain = np.array([[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],[-1.0,4.0],
                   [-1.0,2.0], [0.0,100.0]])


ctrc = ConformalTreeClassification(domain=domain, min_samples_leaf=10, max_depth=24, max_leaves=60, threshold=0.05)

ctrc.calibrate(X_calib, conf_scores, alpha=alpha)

The `test_set` method is used to compute prediction sets for test data. It takes an array of test data of shape $(n,d)$, and a list of length $n$ of dictionaries with class names as keys and assigned probabilities as labels. 

It returns an list of lists, which contain the class names in each prediction set.

In [9]:
conformal_tree_test_sets = ctrc.test_set(X_test, y_test_pred)

ValueError: y_test_pred must be a list

In [ ]:
conformal_tree_test_sets

[['psoriasis'],
 ['psoriasis', 'seboreic dermatitis'],
 ['psoriasis',
  'seboreic dermatitis',
  'lichen planus',
  'pityriasis rosea',
  'cronic dermatitis',
  'pityriasis rubra pilaris'],
 ['psoriasis', 'seboreic dermatitis'],
 ['lichen planus'],
 ['psoriasis', 'seboreic dermatitis', 'pityriasis rosea', 'cronic dermatitis'],
 ['psoriasis',
  'seboreic dermatitis',
  'lichen planus',
  'pityriasis rosea',
  'cronic dermatitis',
  'pityriasis rubra pilaris'],
 ['psoriasis'],
 ['psoriasis'],
 ['psoriasis',
  'seboreic dermatitis',
  'pityriasis rosea',
  'cronic dermatitis',
  'pityriasis rubra pilaris'],
 ['lichen planus'],
 ['psoriasis'],
 ['psoriasis'],
 ['psoriasis', 'seboreic dermatitis', 'pityriasis rosea', 'cronic dermatitis'],
 ['psoriasis', 'seboreic dermatitis'],
 ['psoriasis', 'lichen planus', 'cronic dermatitis'],
 ['psoriasis', 'seboreic dermatitis', 'pityriasis rosea', 'cronic dermatitis'],
 ['psoriasis', 'seboreic dermatitis', 'pityriasis rosea', 'cronic dermatitis'],
 ['